# 中证1000 财务指标因子分析报告

## 分析概述
- **股票池**: 中证1000成分股
- **数据范围**: 2025年1月1日 - 2025年12月31日
- **分析因子**: 18个财务指标
- **分组方法**: Rank分组 (避免极端值问题)

In [ ]:
import pandas as pd
import numpy as np
import json
import os

# 设置路径
results_dir = '/data1/code_git/tick_data_analysis/factor_analysis_results/financial'

# 读取分析结果
with open(f'{results_dir}/indicator_results.json', 'r') as f:
    results = json.load(f)

print(f"分析因子数: {len(results)}")
print(f"数据来源: {results_dir}")

## 1. 因子IC排名

In [ ]:
# 生成因子统计表
stats_data = []
for r in results:
    ic_stats = r.get('ic_stats', {})
    quantile = r.get('quantile_returns', {})
    stats_data.append({
        '因子': r['indicator'],
        'IC均值': ic_stats.get('ic_mean', 0),
        'IC标准差': ic_stats.get('ic_std', 0),
        'IC_IR': ic_stats.get('ic_ir', 0),
        '正IC占比': ic_stats.get('ic_positive_ratio', 0) * 100,
        '多空收益(%)': quantile.get('Long_Short', {}).get('mean_return', 0) * 100 if 'Long_Short' in quantile else 0,
        '股票数': r.get('stock_count', 0),
        '交易日': r.get('date_count', 0)
    })

df_stats = pd.DataFrame(stats_data).sort_values('IC_IR', ascending=False)
df_stats['排名'] = range(1, len(df_stats) + 1)
df_stats = df_stats[['排名', '因子', 'IC均值', 'IC_IR', '正IC占比', '多空收益(%)', '股票数', '交易日']]

print("=" * 100)
print("财务指标因子IC排名 (按IC_IR排序)")
print("=" * 100)
for _, row in df_stats.iterrows():
    print(f"{row['排名']:>2}. {row['因子']:<25} IC={row['IC均值']:>8.4f} IR={row['IC_IR']:>8.4f} 正IC={row['正IC占比']:>6.1f}% 多空={row['多空收益(%)']:>8.2f}%")

df_stats.to_csv(f'{results_dir}/financial_indicator_ic_stats.csv', index=False, encoding='utf-8-sig')

## 2. 按类别统计

In [ ]:
# 按类别统计
categories = {}
for r in results:
    name = r['indicator']
    if '_margin' in name or name in ['roe', 'roa']:
        cat = '盈利能力'
    elif 'ratio' in name and name not in ['pb', 'ps']:
        cat = '偿债能力'
    elif 'turnover' in name:
        cat = '运营效率'
    elif name.startswith('cf_'):
        cat = '现金流'
    elif name in ['pb', 'ps']:
        cat = '估值'
    elif 'growth' in name:
        cat = '成长能力'
    elif 'log_' in name:
        cat = '规模'
    else:
        cat = '其他'
    
    if cat not in categories:
        categories[cat] = []
    ic_ir = r.get('ic_stats', {}).get('ic_ir', 0)
    categories[cat].append(ic_ir)

print("=" * 60)
print("按类别统计")
print("=" * 60)
print(f"{'类别':<15} {'因子数':>8} {'平均IR':>12} {'最佳IR':>12} {'有效因子':>10}")
print("-" * 60)

for cat, irs in sorted(categories.items(), key=lambda x: np.mean(x[1]), reverse=True):
    good = sum(1 for ir in irs if ir > 0.1)
    mean_ir = np.mean(irs)
    best_ir = max(irs)
    print(f"{cat:<15} {len(irs):>8} {mean_ir:>12.4f} {best_ir:>12.4f} {good:>10}")

print("-" * 60)
print("注: 有效因子 = IC_IR > 0.1")

## 3. TOP 10 最佳因子

In [ ]:
print("=" * 80)
print("TOP 10 最佳因子")
print("=" * 80)

for i, r in enumerate(results[:10], 1):
    ic_stats = r.get('ic_stats', {})
    quantile = r.get('quantile_returns', {})
    factor_stats = r.get('factor_stats', {})
    
    print(f"\n{i}. {r['indicator']}")
    print(f"   IC均值: {ic_stats.get('ic_mean', 0):.4f}")
    print(f"   IC_IR: {ic_stats.get('ic_ir', 0):.4f}")
    print(f"   正IC占比: {ic_stats.get('ic_positive_ratio', 0)*100:.1f}%")
    print(f"   因子均值: {factor_stats.get('mean', 0):.4f}")
    
    if 'Long_Short' in quantile:
        print(f"   多空收益: {quantile['Long_Short']['mean_return']*100:.2f}%")

## 4. 分层收益分析

In [ ]:
# 显示主要因子的分层收益
top_factors = ['current_ratio', 'cf_to_nincome', 'gross_margin', 'roe', 'roa']

print("=" * 80)
print("主要因子分层收益")
print("=" * 80)

for r in results:
    if r['indicator'] in top_factors:
        quantile = r.get('quantile_returns', {})
        print(f"\n{r['indicator']}:")
        print(f"   {'分组':<8} {'日均收益':>12} {'标准差':>12} {'样本数':>10}")
        print(f"   {'-' * 44}")
        
        for q in range(1, 6):
            key = f'Q{q}'
            if key in quantile:
                qdata = quantile[key]
                print(f"   {key:<8} {qdata['mean_return']*100:>11.4f}% {qdata['std_return']*100:>11.4f}% {qdata['count']:>10}")
        
        if 'Long_Short' in quantile:
            ls = quantile['Long_Short']
            print(f"   {'多空(Q5-Q1)':<8} {ls['mean_return']*100:>11.4f}%")

## 5. 因子有效性分布

In [ ]:
import matplotlib.pyplot as plt
import matplotlib
matplotlib.rcParams['font.sans-serif'] = ['SimHei', 'DejaVu Sans']
matplotlib.rcParams['axes.unicode_minus'] = False

# 绘制因子IC IR分布
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# IC IR 分布
irs = [r.get('ic_stats', {}).get('ic_ir', 0) for r in results]
names = [r['indicator'] for r in results]

colors = ['green' if ir > 0.1 else 'orange' if ir > 0 else 'red' for ir in irs]

ax1 = axes[0]
bars = ax1.barh(names, irs, color=colors, alpha=0.7)
ax1.axvline(x=0.1, color='green', linestyle='--', label='IR=0.1')
ax1.axvline(x=0, color='black', linestyle='-', linewidth=0.5)
ax1.set_xlabel('IC IR')
ax1.set_title('财务因子 IC IR 分布')
ax1.legend()

# 按类别统计
ax2 = axes[1]
cat_irs = []
cat_names = []
for cat, irs_list in categories.items():
    cat_names.append(cat)
    cat_irs.append(np.mean(irs_list))

sorted_idx = np.argsort(cat_irs)[::-1]
cat_names = [cat_names[i] for i in sorted_idx]
cat_irs = [cat_irs[i] for i in sorted_idx]

ax2.bar(cat_names, cat_irs, color='steelblue', alpha=0.7)
ax2.axhline(y=0.1, color='green', linestyle='--', label='IR=0.1')
ax2.set_ylabel('平均 IC IR')
ax2.set_title('按类别平均 IC IR')
ax2.tick_params(axis='x', rotation=45)
ax2.legend()

plt.tight_layout()
plt.savefig(f'{results_dir}/factor_analysis_summary.png', dpi=150, bbox_inches='tight')
plt.show()

print(f"\n图表已保存: {results_dir}/factor_analysis_summary.png")

## 6. 投资建议总结

In [ ]:
print("=" * 80)
print("投资建议总结")
print("=" * 80)

print("\n【推荐使用的因子】")
print("-" * 60)
for r in results[:5]:
    ic_ir = r.get('ic_stats', {}).get('ic_ir', 0)
    print(f"  • {r['indicator']} (IC_IR = {ic_ir:.4f})")

print("\n【不推荐使用的因子】")
print("-" * 60)
for r in results[-3:]:
    ic_ir = r.get('ic_stats', {}).get('ic_ir', 0)
    print(f"  • {r['indicator']} (IC_IR = {ic_ir:.4f})")

print("\n【关键发现】")
print("-" * 60)
print("""
1. 偿债能力因子（流动比率）表现最佳
2. 盈利能力因子（毛利率、ROA）有一定预测能力
3. 成长因子（营收增长、利润增长）呈负向有效性
4. 估值因子（PB、PS）无明显选股能力

【使用建议】
- 优先使用流动比率、现金流相关因子
- 避免使用成长类因子（均值回归效应）
- 盈利能力因子可作为辅助筛选条件
""")

print("=" * 80)
print("报告生成完成")
print("=" * 80)

---

**注意**: 本分析基于历史数据，仅供参考，不构成投资建议。